In [ ]:
import os, sys, time
from importlib import reload
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../'))
from langchain_anthropic import ChatAnthropic

# Choose llm model
#llm = ChatAnthropic(model="claude-3-5-sonnet-20241022")

# Claude Haiku 3 (legacy) $0.25 $1.25
#llm = ChatAnthropic(model="claude-3-haiku-20240307")
#Claude Haiku 3.5 $0.80 $4
#llm = ChatAnthropic(model="claude-3-5-haiku-20241022")

# Claude Sonnet 4 / 3.7 / 3.5 $3 $15
#llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")
llm = ChatAnthropic(model="claude-sonnet-4-20250514")

##https://console.anthropic.com/settings/billing
# Open‑source (GLM‑4.5 etc)



In [ ]:
import os, time
from lib.lib_drfold import cpy_file
from lib.fasta_utils import read_single_fasta, write_single_fasta
from lib.lib_drfold import clear_output, fasta2pdb

import lib.lib_x3dna
reload(lib.lib_x3dna)
from lib.lib_x3dna import RnaProperties
from lib.lib_x3dna import format_rna_structure

from lib.lib_ribodiffusion import Pdb2Fasta
pdb2fasta = Pdb2Fasta()



c:\Users\User\miniconda3\envs\rna_rag2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model size: 199.2MB


c:\Users\User\work\rna_agent\RiboDiffusion\utils.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_state = torch.load(ckpt_dir, map_location=device)


In [7]:
# Define structured prompt and response schema

from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate

class ResearchResponse(BaseModel):
    topic: str
    summary: str
    sources: list[str]
    tools_used: list[str]
    ancestral_sequence: str
parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a research assistant that will help generate a research paper.
            Answer the user query and use necessary tools. 
            Wrap the output in this format and provide no other text:
            {format_instructions}
            Do not include any other commentary or explanation — output only the JSON.
            """,
        ),
        ("placeholder", "{chat_history}"),
        (
            "human",
            """{query}

Current RNA sequence:
{current_sequence}

3D Structural Parameters of the current RNA:
{rna_structure_block}

Backfolded RNA sequence from 3D structure of the current RNA:
{backfolded_sequence}
""",
        ),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [ ]:
# Create the agent.

from langchain.agents import create_tool_calling_agent, AgentExecutor
from lib.agents.lib_tools import search_tool, save_tool, pubmed_search_tool
from lib.agents.lib_tools import biopython_pubmed_tool, semantic_scholar_tool, europe_pmc_tool

tools = [biopython_pubmed_tool, search_tool, save_tool]
# tools = [biopython_pubmed_tool, save_tool]
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools=tools
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
# Define RNA evolution seed to start the backward evolution simulation

import yaml

# Open and load the JSON file
with open('../configs/rna_rag.yaml', 'r') as f:
    config = yaml.safe_load(f)

# input_dir = "../data/processed/rna_evolution_seed/"
# output_dir = "../data/processed/rna_evolution/"
input_dir = config['input_output']['input_dir']
output_dir = config['input_output']['output_dir_ToolAgent']
os.makedirs(output_dir, exist_ok=True)

# fasta_files = ["fasta_hairpin2"] + [str(i) for i in range(200)]
fasta_files = ["trnaGlycine_Asgard_group_archaeon"] + [str(i) for i in range(1000)]
sequence_0 = read_single_fasta(os.path.join(input_dir, fasta_files[0] + ".fasta"))[1]
print("Initial sequence:", sequence_0)

clear_output(output_dir)
write_single_fasta(
    header=f"seq_{len(sequence_0)}nt",
    sequence=sequence_0,
    filepath=os.path.join(output_dir, fasta_files[0] + ".fasta")
)

allowed_nucleotides = {"A", "U", "G", "C"}
sequence = sequence_0

Initial sequence: GCACCGAUAGUCUAGUGGUAGGACUUAUCCCUGCCUAGGAUAGAGCCCGGGUUCAAAUCCCGGUCGGUGCACCA


In [ ]:
# Loop through each step of the backward evolution process

for i in range(len(fasta_files)-1):
    print("-----> evolution STEP", i)

    # Read FASTA file
    fasta_file = os.path.join(output_dir, fasta_files[i] + ".fasta")
    current_sequence = read_single_fasta(fasta_file)[1]

    print("- Calculating PDB from FASTA") 
    pdb_path = fasta2pdb(source_path = fasta_file)
    #pdb_path_inp = os.path.join(input_dir, fasta_files[0] + ".pdb")
    #pdb_path = fasta_file.replace('.fasta', '.pdb')
    #cpy_file(pdb_path_inp, pdb_path)

    print("- Calculating RNA properties from its 3D structure")
    RNAproperties_3D = RnaProperties(pdb_input=pdb_path)
    helicesInfo = RNAproperties_3D.get_helicesInfo()
    helicesInfo = helicesInfo if isinstance(helicesInfo, dict) else {}
    rna_structure_block = format_rna_structure(RNAproperties_3D.get_genInfo() | helicesInfo)
    
    print()
    print("- Calculating backfolded_rna")
    backfolded_sequences = pdb2fasta.pdb2fasta(source_path = pdb_path, cond_scale=-1.)
    backfolded_sequences = sorted(backfolded_sequences, key=lambda x: x[1], reverse=True) # sort by similarity to current_sequence
    # print(len(backfolded_sequences))
    
    print()
    print("- Running agent")
    index_backfolded_rna = 0
    while index_backfolded_rna < len(backfolded_sequences):
        try:
        #if 1:
            backfolded_sequence, similarity_score = backfolded_sequences[index_backfolded_rna]
            print("---similarity_score:", similarity_score)
            # f"Save both a summary and the proposed ancestral_sequence to a file named 'evolution_step_{i}.txt'. "
            # f"ensuring that the length of the ancestral sequence is less than or equal to the current sequence."
            # f"Append a separator line after the output: -------------------------------"
            query = (
            f"Propose a plausible ancestral RNA sequence based on the current RNA sequence ant its properties."
            f"Save both a summary and the proposed ancestral_sequence to a file" 
            f"and begin each output in the file with the following section header: evolution_step_{i}")
            raw_response = agent_executor.invoke({"query": query,
                                                "current_sequence": current_sequence,
                                                "backfolded_sequence": backfolded_sequence,
                                                "rna_structure_block": rna_structure_block})
            break  # Exit loop if successful
        except Exception as e:
            print(f"Error processing, re trying with another backfolded rna {index_backfolded_rna}: {e}")
            index_backfolded_rna = index_backfolded_rna + 1    
    
    ancestral_sequence = parser.parse(raw_response.get("output")[0]["text"]).ancestral_sequence
    fasta_file_next = output_dir + fasta_files[i+1] + ".fasta"
    write_single_fasta(header="seq", sequence=ancestral_sequence, filepath=fasta_file_next)

-----> evolution STEP 0
- Calculating PDB from FASTA
Copied and renamed to: ../drfold2/file_exchange/fasta_input/fasta.fasta
/opt/drfold2/file_exchange/pdb_output/rets_dir/done is not here. Will generate e2e and geo files.
python /opt/drfold2/cfg_95/test_modeldir.py cuda /opt/drfold2/file_exchange/fasta_input/fasta.fasta /opt/drfold2/file_exchange/pdb_output/rets_dir/cfg_95_ /opt/drfold2/model_hub/cfg_95
python /opt/drfold2/cfg_96/test_modeldir.py cuda /opt/drfold2/file_exchange/fasta_input/fasta.fasta /opt/drfold2/file_exchange/pdb_output/rets_dir/cfg_96_ /opt/drfold2/model_hub/cfg_96
python /opt/drfold2/cfg_97/test_modeldir.py cuda /opt/drfold2/file_exchange/fasta_input/fasta.fasta /opt/drfold2/file_exchange/pdb_output/rets_dir/cfg_97_ /opt/drfold2/model_hub/cfg_97
python /opt/drfold2/cfg_99/test_modeldir.py cuda /opt/drfold2/file_exchange/fasta_input/fasta.fasta /opt/drfold2/file_exchange/pdb_output/rets_dir/cfg_99_ /opt/drfold2/model_hub/cfg_99

Copied and renamed to: ../data/proce

c:\Users\User\miniconda3\envs\rna_rag2\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Invoking: `biopython_pubmed` with `tRNA evolution ancestral reconstruction phylogenetic molecular evolution`
responded: [{'id': 'toolu_015rV5fepV2mgLFqV8FN9SPz', 'input': {}, 'name': 'biopython_pubmed', 'type': 'tool_use', 'index': 0, 'partial_json': '{"__arg1": "tRNA evolution ancestral reconstruction phylogenetic molecular evolution"}'}]

1. 1. J Mol Biol. 2025 Aug 14:169396. doi: 10.1016/j.jmb.2025.169396. Online ahead
of  print.

2. Tracing the Origin of the Genetic Code and Thermostability to Dipeptide 
Sequences in Proteomes.

3. Wang M(1), Aziz MF(1), Caetano-Anollés G(2).

4. Author information:
(1)Department of Crop Sciences and Carl R. Woese Institute for Genomic Biology, 
University of Illinois at Urbana-Champaign, Urbana, IL 61801, USA.
(2)Department of Crop Sciences and Carl R. Woese Institute for Genomic Biology, 
University of Illinois at Urbana-Champaign, Urbana, IL 61801, USA. Electronic 
address: gca@illinois.edu.

5. The safekeeping of the genetic code has been entr

c:\Users\User\miniconda3\envs\rna_rag2\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Dec 21, 2024 · Abstract We introduce the 12th version of the Molecular Evolutionary Genetics Analysis (MEGA12) software. This latest version brings many significant improvements by … 4 days ago · To answer these questions, the researchers have developed a new bioinformatics-based method, which they present in Molecular Biology and Evolution. 18,000 species in the … Aug 20, 2025 · 2.3 Approaches 3: evolutionary-based exploration In these approaches, we explore the potential for mutual collaboration between reinforcement learning and evolutionary … Aug 14, 2025 · Molecular Plant 8:1274â€“1284. 867 Manichaikul A, Mychaleckyj JC, Rich SS, Daly K, Sale M, Chen W-M. 2010. Robust relationship inference 868 in genome-wide … Dec 25, 2024 · This evolution can introduce discrepancies in enrichment analysis outcomes when different ontology versions are applied. A systematic analysis highlighted low consistency …
Invoking: `biopython_pubmed` with `RNA sequence evolution ancestral state reconstructi

...

c:\Users\User\miniconda3\envs\rna_rag2\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


但是，RNA和DNA在组成上的主要差别是：①RNA的糖分子是核糖，DNA的是脱氧核糖，②四种碱基中有一种不同。 所以让我们来总结一下他们的物质关系：染 … 标准 的RNA电泳图，条带是2-3条（这样说可能不是太严谨），提取RNA的质量比较好应该是主要看见的是两条带。从上到下依次是28s，18s和5s。其中主要 … 概述核糖核酸（英语：ribonucleic acid，缩写：RNA），是一类由核糖核苷酸通过磷酸二酯键聚合而成的线性大分子。自然界中的RNA通常是单链的， … 非编码RNA是指不直接参与蛋白质合成的RNA，它们在细胞内具有调控基因表达、稳定基因组、调控细胞周期功能。 非编码RNA可分为非编码大RNA和非编码 … RNA化学结构，5`端，3`端及五碳糖1号碳位置均标出 从三级结构上来说，DNA为双螺旋结构，RNA根据功能不同有多种结构。如tRNA的三叶草型，mRNA的单链 …
Invoking: `search` with `RNA evolution ancestral reconstruction computational methods`
responded: [{'id': 'toolu_01LaBnUoDiod2jrCAFGgbVhy', 'input': {}, 'name': 'search', 'type': 'tool_use', 'index': 0, 'partial_json': '{"__arg1": "RNA evolution ancestral reconstruction computational methods"}'}]



c:\Users\User\miniconda3\envs\rna_rag2\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


No good DuckDuckGo Search Result was found
Invoking: `biopython_pubmed` with `RNA secondary structure evolution phylogenetic reconstruction`
responded: [{'id': 'toolu_01YMES4EbY3gPJqGdeMwdxGK', 'input': {}, 'name': 'biopython_pubmed', 'type': 'tool_use', 'index': 0, 'partial_json': '{"__arg1": "RNA secondary structure evolution phylogenetic reconstruction"}'}]

1. 1. Nucleic Acids Res. 2025 Aug 11;53(15):gkaf761. doi: 10.1093/nar/gkaf761.

2. Presence of group II introns in phage genomes.

3. Merk LN(1)(2)(3), Jones TA(1)(2), Eddy SR(1)(2).

4. Author information:
(1)Department of Molecular and Cellular Biology, Harvard University, Cambridge, 
MA 02138, United States.
(2)Howard Hughes Medical Institute, Harvard University, Cambridge, MA 02138, 
United States.
(3)Harvard Graduate Program in Biophysics, Harvard University, Cambridge, MA 
02138, United States.

5. Update of
    bioRxiv. 2025 May 23:2025.05.22.655115. doi: 10.1101/2025.05.22.655115.

6. Although bacteriophage genomes are u

FileNotFoundError: [Errno 2] No such file or directory: '../drfold2/file_exchange/pdb_output/relax/model_1.pdb'

In [ ]:
print(parser.parse(raw_response.get("output")[0]["text"]).summary)

In [ ]:
print(current_sequence)
ancestral_sequence = parser.parse(raw_response.get("output")[0]["text"]).ancestral_sequence
print(ancestral_sequence)

In [ ]:
# Compare two RNA sequences

from Bio.Align import PairwiseAligner

# Example RNA sequences
rna1 = current_sequence 
rna2 = ancestral_sequence 



# Create and Configure the Aligner
aligner = PairwiseAligner()

# Optional: adjust scoring
aligner.match_score = 2
aligner.mismatch_score = -1
aligner.open_gap_score = -2
aligner.extend_gap_score = -0.5

# Run the Alignment
alignments = aligner.align(rna1, rna2)

# Print the Best Alignment
best_alignment = alignments[0]
print(type(best_alignment))
print(best_alignment)
print(f"Score: {best_alignment.score}")

